In [23]:
%matplotlib inline

import numpy as np
from nibabel.affines import apply_affine, from_matvec
from nibabel.eulerangles import euler2mat
from matplotlib import pyplot as plt
from matplotlib import colormaps as cm
from matplotlib import colors
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D

import pyvista as pv


plt.rcParams['font.family'] = "Libre Franklin"
plt.rcParams['text.usetex'] = False

In [ ]:
# Add convenient 3D annotations
# credits: https://gist.github.com/WetHat/1d6cd0f7309535311a539b42cccca89c
from matplotlib.text import Annotation

class Annotation3D(Annotation):

    def __init__(self, text, xyz, *args, **kwargs):
        super().__init__(text, xy=(0, 0), *args, **kwargs)
        self._xyz = xyz

    def draw(self, renderer):
        x2, y2, z2 = proj_transform(*self._xyz, self.axes.M)
        self.xy = (x2, y2)
        super().draw(renderer)
        


class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 

    
def _annotate3D(ax, text, xyz, *args, **kwargs):
    '''Add anotation `text` to an `Axes3d` instance.'''

    annotation = Annotation3D(text, xyz, *args, **kwargs)
    ax.add_artist(annotation)

setattr(Axes3D, 'annotate3D', _annotate3D)

def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)


setattr(Axes3D, 'arrow3D', _arrow3D)

In [24]:
# Color data
cmap = cm["viridis"]

def get_color(value, alpha=1.0):
    return colors.to_hex((*cmap(value)[:3], alpha), keep_alpha=True)

vect_color = np.vectorize(get_color)

In [25]:
def explode(data):
    """Create gaps between voxels."""
    size = np.array(data.shape) * 2
    data_e = np.zeros(size - 1, dtype=data.dtype)
    data_e[::2, ::2, ::2] = data
    return data_e

In [26]:
shape = (5, 4, 3)
n_voxels = np.arange(1, np.prod(shape) + 1).reshape(shape)

filled = explode(n_voxels)
fcolor = explode(vect_color(n_voxels / n_voxels.size, alpha=0.8))

# Shrink the gaps
i, j, k = np.indices(np.array(filled.shape) + 1).astype(float) // 2
i[0::2, :, :] += 0.05
j[:, 0::2, :] += 0.05
k[:, :, 0::2] += 0.05
i[1::2, :, :] += 0.95
j[:, 1::2, :] += 0.95
k[:, :, 1::2] += 0.95

In [ ]:
def plot_cube(x, y, z, filled, facecolors, zorder=0):
    ax = plt.figure(figsize=(20, 20)).add_subplot(projection='3d')
    ax.voxels(x, y, z, filled, facecolors=facecolors, zorder=0)
    ax.set_aspect('equal')
    ax.grid(False)
    plt.axis("off")
    
    ax.arrow3D(
        -0.5, -0.2, -0.2,
        n_voxels.shape[0] + 1.2, 0.00, 0.0,
        arrowstyle="-|>",
        mutation_scale=60,
        fc="k",
        lw=7
    )

    ax.arrow3D(
        -0.2, -0.2, -0.5,
        -0.0, -0.0, n_voxels.shape[2] + 1.2,
        arrowstyle="-|>",
        mutation_scale=60,
        fc="k",
        lw=7
    )

    ax.annotate3D(
        "$i$-axis",
        (n_voxels.shape[0] - 0.5, 0, 0),
        xytext=(0, -70),
        size=50,
        textcoords='offset points',
        horizontalalignment="center",
        verticalalignment="top",
        zorder=100000000,
        annotation_clip=False,
    )

    ax.annotate3D(
        "$k$-axis",
        (0, 0, n_voxels.shape[2]),
        xytext=(-60, -10),
        size=50,
        textcoords='offset points',
        horizontalalignment="center",
        verticalalignment="top",
        zorder=100000000,
        rotation=90,
        annotation_clip=False,
    )
    
    return ax

In [ ]:
def update_alpha(value, alpha=1.0):
    acode = "{:02x}".format(int(255 * alpha))
    return f"{value[:8]}{acode}"

vect_alpha = np.vectorize(update_alpha)

In [ ]:
ANNOTATIONS = {
    "(0, 0, 0)": (
        (0.5, 0.0, 0.5),
        (-60, -140),
    ),
    "(0, 0, 1)": (
        (0.5, 0.5, 2),
        (-60, 140),
    ),
    "(0, 0, 2)": (
        (0.5, 0.5, n_voxels.shape[2]),
        (-60, 140),
    ),
    "(0, 1, 0)": (
        (0.5, 1.5, 1),
        (-60, 140),
    ),
    "(0, 3, 0)": (
        (0.5, n_voxels.shape[1] - 0.5, 1),
        (-60, 140),
    ),
    "(0, 3, 1)": (
        (0.5, n_voxels.shape[1] - 0.5, 2),
        (-60, 140),
    ),
    "(0, 3, 2)": (
        (0.5, n_voxels.shape[1] - 0.5, n_voxels.shape[2]),
        (-60, 140),
    ),
    "(4, 0, 0)": (
        (n_voxels.shape[0], 0.5, 0.5),
        (60, -80),
    ),
    "(4, 1, 0)": (
        (n_voxels.shape[0] - 0.5, 1.5, 1),
        (60, 140),
    ),
    "(4, 3, 0)": (
        (n_voxels.shape[0], n_voxels.shape[1] - 0.5, 0.5),
        (60, -80),
    ),
    "(4, 3, 1)": (
        np.array(n_voxels.shape) - (0.5, 0.5, 1),
        (60, 140),
    ),
    "(4, 3, 2)": (
        np.array(n_voxels.shape) - (0.5, 0.5, 0),
        (60, 140),
    ),
}

def annotate_coords(ax, coords, label=None):
    pos = ANNOTATIONS[coords]
    ax.annotate3D(
        label or coords,
        pos[0],
        xytext=pos[1],
        size=25,
        color="dimgray",
        textcoords='offset points',
        horizontalalignment="center",
        verticalalignment="top",
        arrowprops=dict(
            arrowstyle="fancy",
            fc='lightgray',
            ec='lightgray',
            lw=3,
            shrinkA=10,
        ),
        zorder=100000000,
    )

In [ ]:
n_voxels_values = n_voxels.copy()

ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)")
annotate_coords(ax, "(4, 0, 0)")
annotate_coords(ax, "(0, 0, 2)")
annotate_coords(ax, "(0, 3, 2)")
annotate_coords(ax, "(4, 3, 0)")
annotate_coords(ax, "(4, 3, 2)")

plt.savefig("00-voxels.png", dpi=300)

In [ ]:
ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)", label=f"{n_voxels_values[0, 0, 0]}")
annotate_coords(ax, "(4, 0, 0)", label=f"{n_voxels_values[4, 0, 0]}")
annotate_coords(ax, "(0, 0, 2)", label=f"{n_voxels_values[0, 0, 2]}")
annotate_coords(ax, "(0, 3, 2)", label=f"{n_voxels_values[0, 3, 2]}")
annotate_coords(ax, "(4, 3, 0)", label=f"{n_voxels_values[4, 3, 0]}")
annotate_coords(ax, "(4, 3, 2)", label=f"{n_voxels_values[4, 3, 2]}")

plt.savefig("01-voxels.png", dpi=300)

In [ ]:
n_voxels_values = n_voxels.copy()
n_voxels_values[:, :, 1:] = 0
n_voxels_values[:, 1:, :] = 0

ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)")
annotate_coords(ax, "(4, 0, 0)")

plt.savefig("02-voxels.png", dpi=300)

In [ ]:
ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)", label=f"{n_voxels_values[0, 0, 0]}")
annotate_coords(ax, "(4, 0, 0)", label=f"{n_voxels_values[4, 0, 0]}")

plt.savefig("03-voxels.png", dpi=300)

In [ ]:
n_voxels_values = n_voxels.copy()
n_voxels_values[:, :, 1:] = 0
n_voxels_values[:, 2:, :] = 0

ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)")
annotate_coords(ax, "(0, 1, 0)")
annotate_coords(ax, "(4, 0, 0)")
annotate_coords(ax, "(4, 1, 0)")
plt.savefig("04-voxels.png", dpi=300)

In [ ]:
ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)", label=f"{n_voxels_values[0, 0, 0]}")
annotate_coords(ax, "(0, 1, 0)", label=f"{n_voxels_values[0, 1, 0]}")
annotate_coords(ax, "(4, 0, 0)", label=f"{n_voxels_values[4, 0, 0]}")
annotate_coords(ax, "(4, 1, 0)", label=f"{n_voxels_values[4, 1, 0]}")

plt.savefig("05-voxels.png", dpi=300)

In [ ]:
n_voxels_values = n_voxels.copy()
n_voxels_values[:, :, 1:] = 0

ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)")
annotate_coords(ax, "(4, 0, 0)")
annotate_coords(ax, "(0, 3, 0)")
annotate_coords(ax, "(4, 3, 0)")

plt.savefig("06-voxels.png", dpi=300)

In [ ]:
ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)", label=f"{n_voxels_values[0, 0, 0]}")
annotate_coords(ax, "(4, 0, 0)", label=f"{n_voxels_values[4, 0, 0]}")
annotate_coords(ax, "(0, 3, 0)", label=f"{n_voxels_values[0, 3, 0]}")
annotate_coords(ax, "(4, 3, 0)", label=f"{n_voxels_values[4, 3, 0]}")

plt.savefig("07-voxels.png", dpi=300)

In [ ]:
n_voxels_values = n_voxels.copy()
n_voxels_values[:, :, 2:] = 0

ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)")
annotate_coords(ax, "(4, 0, 0)")
annotate_coords(ax, "(0, 0, 1)")
annotate_coords(ax, "(0, 3, 1)")
annotate_coords(ax, "(4, 3, 0)")
annotate_coords(ax, "(4, 3, 1)")

plt.savefig("08-voxels.png", dpi=300)

In [ ]:
ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)", label=f"{n_voxels_values[0, 0, 0]}")
annotate_coords(ax, "(4, 0, 0)", label=f"{n_voxels_values[4, 0, 0]}")
annotate_coords(ax, "(0, 0, 1)", label=f"{n_voxels_values[0, 0, 1]}")
annotate_coords(ax, "(0, 3, 1)", label=f"{n_voxels_values[0, 3, 1]}")
annotate_coords(ax, "(4, 3, 0)", label=f"{n_voxels_values[4, 3, 0]}")
annotate_coords(ax, "(4, 3, 1)", label=f"{n_voxels_values[4, 3, 1]}")

plt.savefig("09-voxels.png", dpi=300)

In [ ]:
n_voxels_values = n_voxels.copy()
n_voxels_values = np.flip(n_voxels_values, 0)

ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)")
annotate_coords(ax, "(4, 0, 0)")
annotate_coords(ax, "(0, 0, 2)")
annotate_coords(ax, "(0, 3, 2)")
annotate_coords(ax, "(4, 3, 0)")
annotate_coords(ax, "(4, 3, 2)")

plt.savefig("10-voxels.png", dpi=300)

In [ ]:
ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

annotate_coords(ax, "(0, 0, 0)", label=f"{n_voxels_values[0, 0, 0]}")
annotate_coords(ax, "(4, 0, 0)", label=f"{n_voxels_values[4, 0, 0]}")
annotate_coords(ax, "(0, 0, 2)", label=f"{n_voxels_values[0, 0, 2]}")
annotate_coords(ax, "(0, 3, 2)", label=f"{n_voxels_values[0, 3, 2]}")
annotate_coords(ax, "(4, 3, 0)", label=f"{n_voxels_values[4, 3, 0]}")
annotate_coords(ax, "(4, 3, 2)", label=f"{n_voxels_values[4, 3, 2]}")

plt.savefig("11-voxels.png", dpi=300)

In [ ]:
n_voxels_values = np.zeros_like(n_voxels)
n_voxels_values[1, :, :] = n_voxels[1, :, :]
n_voxels_values[:, 2, :] = n_voxels[:, 2, :]
n_voxels_values[:, :, 1] = n_voxels[:, :, 1]

ax = plot_cube(
    i, j, k,
    explode(n_voxels_values),
    facecolors=explode(vect_color(n_voxels_values / n_voxels_values.size, alpha=0.8)),
)

ax.annotate3D(
    "sagittal slice ($i$ = 1)",
    (1.5, n_voxels.shape[1] - 0.5, n_voxels.shape[2]),
    xytext=(60, 120),
    size=25,
    color="dimgray",
    textcoords='offset points',
    horizontalalignment="center",
    verticalalignment="top",
    arrowprops=dict(
        arrowstyle="fancy",
        fc='lightgray',
        ec='lightgray',
        lw=3,
        shrinkA=10,
    ),
    zorder=100000000,
)

ax.annotate3D(
    "coronal slice ($j$ = 2)",
    (0.5, n_voxels.shape[1] - 1.5, n_voxels.shape[2]),
    xytext=(-60, 120),
    size=25,
    color="dimgray",
    textcoords='offset points',
    horizontalalignment="center",
    verticalalignment="top",
    arrowprops=dict(
        arrowstyle="fancy",
        fc='lightgray',
        ec='lightgray',
        lw=3,
        shrinkA=10,
    ),
    zorder=100000000,
)

ax.annotate3D(
    "axial slice ($k$ = 1)",
    (n_voxels.shape[0] - 0.5, 0, 1.5),
    xytext=(-50, -140),
    size=25,
    color="dimgray",
    textcoords='offset points',
    horizontalalignment="center",
    verticalalignment="top",
    arrowprops=dict(
        arrowstyle="fancy",
        fc='lightgray',
        ec='lightgray',
        lw=3,
        shrinkA=10,
    ),
    zorder=100000000,
)

plt.savefig("12-voxels.png", dpi=300)

In [ ]:
def heatmap(data, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
#     cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
#     cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ticks = np.arange(data.shape[-1])[2::3]
    ax.set_xticks(ticks, labels=data[-1, ticks])
#     ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

#     # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), ha="center",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[-1]) - .5, minor=True)
    ax.set_yticks([])
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im


In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
im = heatmap(n_voxels.ravel()[np.newaxis, :], ax=ax)

for j, k in enumerate(n_voxels.ravel()[0::n_voxels.shape[-1]]):
    ax.annotate(f'$j$ = {j % n_voxels.shape[1]}', xy=(k, 1), xytext=(k, 2.2),
                fontsize=14, ha='center', va='bottom', xycoords='data', 
                arrowprops=dict(arrowstyle='-[, widthB=1.8, lengthB=.4', lw=2.0), annotation_clip=False)

j_length = int(np.prod(n_voxels.shape[-2:]))
j_offset = 0.5 * (j_length - 1)
for i, j in enumerate(range(0, n_voxels.size, j_length)):
    ax.annotate(f'$i$ = {i}', xy=(j + j_offset, 2.4), xytext=(j + j_offset, 3.6),
                fontsize=14, ha='center', va='bottom', xycoords='data', 
                arrowprops=dict(arrowstyle='-[, widthB=7.8, lengthB=.4', lw=2.0), annotation_clip=False)

plt.savefig("13-voxels.png", dpi=300)

In [27]:
# Create the spatial reference
grid = pv.ImageData()

# Set the grid dimensions: shape because we want to inject our values on the
#   CELL data
grid.dimensions = np.array(n_voxels.shape) + 1

# Edit the spatial reference
grid.origin = (0, 0, 0)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.cell_data["values"] = n_voxels.flatten()  # Flatten the array

p = pv.Plotter()
p.add_mesh(grid, show_edges=True, show_scalar_bar=False)
p.camera.position = (12.664973128311377, -16.551422795669886, 9.491752120601783)
p.camera.zoom(1.2)
p.show_grid(xtitle="i-axis", ytitle="j-axis", ztitle="k-axis")
p.export_html("voxels-grid.html")
p.close()

In [ ]:
affine = from_matvec(np.diag((1.0, 1.0, 2.0)), (0, 0, 0))

# Create an unstructured grid
unstruct = grid.cast_to_unstructured_grid()

# Transform
unstruct.points = apply_affine(affine, grid.points)

p = pv.Plotter()
p.add_mesh(unstruct, show_edges=True, show_scalar_bar=False)
p.camera.position = (12.664973128311377, -16.551422795669886, 9.491752120601783)
p.camera.zoom(1.2)
p.show_grid(xtitle="x-axis", ytitle="y-axis", ztitle="z-axis")
p.export_html("voxels-grid-spacing.html")
p.close()

In [ ]:
affine = from_matvec(euler2mat(z=0.0, x=0.0, y=np.pi / 4), (0, 0, 0)) * (1.0, 1.0, 2.0, 1.0)
origin = -0.5 * apply_affine(affine, np.array(n_voxels.shape))
affine[:3, 3] = origin

# Create an unstructured grid
unstruct = grid.cast_to_unstructured_grid()

# Transform
unstruct.points = apply_affine(affine, grid.points)

# Plot
p = pv.Plotter()
p.add_mesh(unstruct, show_edges=True, show_scalar_bar=False)
p.camera.position = (12.81974682389558, -17.213299341874844, 7.475116926921199)
p.camera.zoom(0.8)
p.show_grid(xtitle="x-axis", ytitle="y-axis", ztitle="z-axis")
p.export_html("voxels-grid-spacing-rotation-y.html")
p.close()

In [ ]:
affine = from_matvec(euler2mat(z=0.4, x=0.0, y=np.pi / 4), (0, 0, 0)) * (1.0, 1.0, 2.0, 1.0)
origin = -0.5 * apply_affine(affine, np.array(n_voxels.shape))
affine[:3, 3] = origin

# Create an unstructured grid
unstruct = grid.cast_to_unstructured_grid()

# Transform
unstruct.points = apply_affine(affine, grid.points)

# Plot
p = pv.Plotter()
p.add_mesh(unstruct, show_edges=True, show_scalar_bar=False)
p.camera.position = (12.81974682389558, -17.213299341874844, 7.475116926921199)
p.camera.zoom(0.8)
p.show_grid(xtitle="x-axis", ytitle="y-axis", ztitle="z-axis")
p.export_html("voxels-grid-spacing-rotation-y-z.html")
p.close()

In [29]:
affine = from_matvec(euler2mat(z=0.4, x=0.0, y=np.pi / 4), (0, 0, 0)) * (-1.0, 1.0, 2.0, 1.0)
origin = -0.5 * apply_affine(affine, np.array(n_voxels.shape))
affine[:3, 3] = origin

# Create an unstructured grid
unstruct = grid.cast_to_unstructured_grid()

# Transform
unstruct.points = apply_affine(affine, grid.points)

# Plot
p = pv.Plotter()
p.add_mesh(unstruct, show_edges=True, show_scalar_bar=False)
p.camera.position = (12.81974682389558, -17.213299341874844, 7.475116926921199)
p.camera.zoom(0.8)
p.show_grid(xtitle="x-axis", ytitle="y-axis", ztitle="z-axis")
p.export_html("voxels-grid-flip-spacing-rotation-y-z.html")
p.close()

In [2]:
import nibabel as nb
# Load a NIfTI file
ni_img = nb.load("/data/datasets/hcph/sub-001/ses-001/anat/sub-001_ses-001_acq-undistorted_T1w.nii.gz")

# Convert NIfTI data to pyvista mesh
data = np.asanyarray(nb.as_closest_canonical(ni_img).dataobj)

# Create interactive plotter ()
p = pv.Plotter()
p.add_volume(data, cmap="gray", show_scalar_bar=False)
p.show_grid(xtitle="i-axis", ytitle="j-axis", ztitle="k-axis")
# p.export_html("brain.html")
p.close()

In [30]:
import nibabel as nb
# Load a NIfTI file
ni_img = nb.load("/data/datasets/hcph/sub-001/ses-001/anat/sub-001_ses-001_acq-undistorted_T1w.nii.gz")

# Convert NIfTI data to pyvista mesh
data = np.flip(np.asanyarray(nb.as_closest_canonical(ni_img).dataobj), 1)

# Create interactive plotter ()
p = pv.Plotter()
p.add_volume(data, cmap="gray", show_scalar_bar=False)
p.show_grid(xtitle="i-axis", ytitle="j-axis", ztitle="k-axis")
p.export_html("brain-flip-j.html")
p.close()

In [22]:
last_camera_pos = p.camera.position
affine = ni_img.affine

spacing = nb.affines.voxel_sizes(affine)[:3]


# Create the spatial reference
grid = pv.ImageData(np.asanyarray(ni_img.dataobj).flatten("F"))

# Set the grid dimensions: shape because we want to inject our values on the
#   CELL data
# grid.dimensions = np.array(ni_img.dataobj.shape) + 1

# Edit the spatial reference
# grid.origin = affine[:3, 3]  # The bottom left corner of the data set
grid.spacing = spacing  # These are the cell sizes along each axis

# Add the data values to the cell data
# grid.cell_data["values"] = np.asanyarray(ni_img.dataobj).flatten("F")  # Flatten the array

# z, y, x = nb.eulerangles.mat2euler(affine[:3, :3])
z, y, x = (0, 0, 0)

# unstruct = grid.rotate_z(z)
# unstruct = unstruct.rotate_y(y)
# unstruct = unstruct.rotate_x(x)
# unstruct = grid.translate(affine[:3, 3])

# Create an unstructured grid
# unstruct = grid.cast_to_unstructured_grid()

# Transform
# unstruct.points = apply_affine(ni_img.affine, grid.points)

# unstruct = grid.transform(affine, inplace=False)

p = pv.Plotter()
# p.add_volume(grid.rotate_z(z).rotate_y(y).rotate_x(x), cmap="gray", show_scalar_bar=False)
# p.add_volume(grid.transform(np.eye(4), inplace=False), cmap="gray", show_scalar_bar=False)
p.add_volume(grid, cmap="gray", show_scalar_bar=False)
p.show_grid(xtitle="x-axis", ytitle="y-axis", ztitle="z-axis")
# p.camera.position = last_camera_pos
p.export_html("brain-affine.html")
p.close()

TypeError: First argument, ``uinput`` must be either ``vtk.vtkImageData`` or a path, not <class 'numpy.ndarray'>.  Use keyword arguments to specify dimensions, spacing, and origin. For example:

    >>> grid = pv.ImageData(
    ...     dimensions=(10, 10, 10),
    ...     spacing=(2, 1, 5),
    ...     origin=(10, 35, 50),
    ... )
